In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#以票價 Fare 作為數值型欄位，Cabin 作為類別型欄位

df['Cabin'] = df['Cabin'].fillna('None')
mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x:x.mode()[0]).reset_index()
median_df = df.groupby(['Cabin'])['Fare'].median().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
temp.columns = ['Cabin','Fare_mean','Fare_mode','Fare_median','Fare_max']
df = pd.merge(df,temp,on=['Cabin'])
df = df.drop(['Cabin'],axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.05,10.5,512.3292
1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.05,10.5,512.3292
2,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.05,10.5,512.3292
3,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q,19.157325,8.05,10.5,512.3292
4,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,19.157325,8.05,10.5,512.3292


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,22.0,1,0,7.2500,19.157325,8.05,10.5,512.3292
1,3,26.0,0,0,7.9250,19.157325,8.05,10.5,512.3292
2,3,35.0,0,0,8.0500,19.157325,8.05,10.5,512.3292
3,3,-1.0,0,0,8.4583,19.157325,8.05,10.5,512.3292
4,3,2.0,3,1,21.0750,19.157325,8.05,10.5,512.3292


In [4]:
# 原始特徵
df_minus = df.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max'] , axis=1)

# 原始特徵 + 羅吉斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6127940514655894

In [5]:
# 新特徵 + 羅吉斯迴歸 : 有改善
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6150475904740322

In [6]:
# 原始特徵 + 梯度提升樹
train_X = MMEncoder.fit_transform(df_minus)
estimator = GradientBoostingRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

-0.13092400871629364

In [7]:
# 新特徵 + 梯度提升樹
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

-0.1822775803555235